# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [12]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import holoviews as hv
import matplotlib.pyplot as plt

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ushuaia,-54.8000,-68.3000,7.81,71,0,4.12,AR,1709514381
1,1,adamstown,-25.0660,-130.1015,24.84,69,2,6.89,PN,1709514648
2,2,isiro,2.7739,27.6160,21.06,90,38,1.57,CD,1709514648
3,3,dongola,19.1667,30.4833,19.09,14,87,4.85,SD,1709514648
4,4,arnedo,42.2167,-2.1000,2.72,82,17,2.01,ES,1709514648


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display
# Adjust the size of the points to be 50% of the humidity value
point_size = city_data_df["Humidity"] * 0.5
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    x="Lng",
    y="Lat",
    geo=True,
    tiles="OSM",
    size=point_size,
    color="Humidity",
    cmap="viridis",
    title="City Map with Humidity",
    hover_cols=["City", "Country", "Max_Temp", "Humidity"]
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,_size,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# Define the list of desired countries
desired_countries = ["FR", "ES", "JP", "AU", "MA"]

# Modify the ideal_conditions to use isin for country filtering
ideal_conditions = (
    city_data_df['Country'].isin(desired_countries) & 
    (city_data_df["Cloudiness"] < 25) & 
    (city_data_df["Wind Speed"] < 7)
)

# Narrow down the DataFrame using the ideal conditions
ideal_weather_df = city_data_df[ideal_conditions]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,arnedo,42.2167,-2.1000,2.72,82,17,2.01,ES,1709514648
80,80,la passe,45.5549,-0.8967,2.13,71,6,1.63,FR,1709514660
87,87,hasaki,35.7333,140.8333,11.27,41,0,6.18,JP,1709514661
134,134,mount isa,-20.7333,139.5000,32.87,49,11,3.60,AU,1709514673
153,153,paris,48.8534,2.3488,4.52,89,0,3.09,FR,1709514662


In [6]:
map_plot_2 = ideal_weather_df.hvplot.points(
    x="Lng",
    y="Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",  # You can choose a different size metric if needed
    color="Humidity",  # You can choose a different color metric if needed
    cmap="viridis",  # You can choose a different colormap if needed
    title="Cities with Ideal Weather Conditions",
    hover_cols=["City", "Country", "Max Temp (F)", "Cloudiness", "Wind Speed"]
)

# Display the map for ideal weather conditions
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Cloudiness,Wind Speed)

### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create a DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
4,arnedo,ES,42.2167,-2.1000,82,
80,la passe,FR,45.5549,-0.8967,71,
87,hasaki,JP,35.7333,140.8333,41,
134,mount isa,AU,-20.7333,139.5000,49,
153,paris,FR,48.8534,2.3488,89,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
 # Set the parameters for the type of place (e.g., "hotel" for your case)
categories = "accommodation.hotel"

# Set the parameters for the type of search
radius = 10000 
limit = 1  

# Set up a parameters dictionary
params = {
    "categories": categories,
    "limit": limit,
    "apiKey": geoapify_key
}

# Set base URL for the find endpoint
base_url = "https://api.geoapify.com/v2/places"

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Set the filter and bias parameters based on the current city's latitude and longitude
    filters = f"circle:{lng},{lat},{radius}"
    bias = f"proximity:{lng},{lat}"
    
    # Update the params dictionary with the current filter and bias parameters
    params["filter"] = filters
    params["bias"] = bias
    
    # Run a request using the params dictionary
    response = requests.get(base_url, params=params).json()

    try:
        hotel_df.loc[index, "Hotel Name"] = response["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

arnedo - nearest hotel: Hotel Victoria
la passe - nearest hotel: L'Estuaire
hasaki - nearest hotel: Choshi Plaza Hotel
mount isa - nearest hotel: Ibis Styles
paris - nearest hotel: Hôtel Esmerelda
mount gambier - nearest hotel: The Old Mount Gambier Gaol
codrington - nearest hotel: No hotel found
new norfolk - nearest hotel: Woodbridge on the Derwent
le grau-du-roi - nearest hotel: Hôtel Quai D´Azur
casablanca - nearest hotel: Hotel de Prince de Paris
broken hill - nearest hotel: Ibis Styles
kushima - nearest hotel: No hotel found
ulladulla - nearest hotel: Hotel Marlin
warrnambool - nearest hotel: Central Court
saint-francois - nearest hotel: Chez Lily
hamamatsu - nearest hotel: 浜松ホテル
parkes - nearest hotel: Commercial Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
4,arnedo,ES,42.2167,-2.1000,82,Hotel Victoria
80,la passe,FR,45.5549,-0.8967,71,L'Estuaire
87,hasaki,JP,35.7333,140.8333,41,Choshi Plaza Hotel
134,mount isa,AU,-20.7333,139.5000,49,Ibis Styles
153,paris,FR,48.8534,2.3488,89,Hôtel Esmerelda


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    x="Lng",
    y="Lat",
    geo=True,
    tiles="OSM",
    size="Humidity",
    color="Humidity",
    cmap="viridis",
    title="City Map with Hotel Information",
    hover_cols=["City", "Country", "Humidity", "Hotel Name"]
)


# Display the map
hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)

<Figure size 640x480 with 0 Axes>